In [40]:
# -*- coding: utf8 -*-

In [41]:
debug=False

filePrefix="MGC-160104"
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync2.txt"
nomConverts="/Users/gilles/Github/SWIM/2015-Data/VerbesAction-ConvertsPP.txt"
nomAutres="/Users/gilles/Github/SWIM/2015-Data/VerbesAction-Autres.txt"

freqForme=1000000

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [83]:
import pandas as pd
import pickle
import numpy as np

#Préparation du tableau de VERBES

###Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [43]:
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")
bdlexique.loc[(bdlexique["ms"]=="ij")&(bdlexique["cs"].isin(["V","K"])),"ms"]="MS"
bdlexique.drop_duplicates(inplace=True,subset=["phono","ext","cs","ms","vs","lexeme","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"])

### Création de la colonne de fréquence *freq*

- fréquence forme : 1000000x(formeFrantext+formeFilms)
- fréquence lexeme : lemmeFrantext+lemmeFilms
- fréquence inexistante : .01

In [44]:
bdlexique["freq"]=1.0/100
filtreLFrantext=(bdlexique["lemmeFrantext"]!="***") 
filtreLFilms=(bdlexique["lemmeFilms"]!="***")
filtreFFrantext=(bdlexique["formeFrantext"]!="***") 
filtreFFilms=(bdlexique["formeFilms"]!="***")
bdlexique.loc[filtreLFrantext|filtreLFilms,"freq"]=bdlexique[filtreLFrantext]["lemmeFrantext"].astype(float)+bdlexique[filtreLFilms]["lemmeFilms"].astype(float)
bdlexique.loc[filtreFFrantext | filtreFFrantext,"freq"]=bdlexique[filtreFFrantext]["formeFrantext"].astype(float)*freqForme+bdlexique[filtreFFilms]["formeFilms"].astype(float)*freqForme

### Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [45]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

In [ ]:
convertsPP=pd.read_csv(nomConverts,sep=";",names=["lexeme","base"],encoding="utf8")
autresDerives=pd.read_csv(nomAutres,sep=";",names=["lexeme","base"],encoding="utf8")


In [69]:
derivesLexicaux=pd.concat([convertsPP,autresDerives]).reset_index()
del derivesLexicaux["index"]
derivesLexicaux=derivesLexicaux.drop_duplicates()
listDerives=derivesLexicaux["lexeme"].tolist()

### Extraction des converts dans une structure *derives*

In [73]:
derives=bdlexique[(bdlexique["cs"]=="N") & (bdlexique["ms"].str.startswith("F")) & (bdlexique["lexeme"].isin(listDerives))]

In [74]:
derives=derives.merge(derivesLexicaux,on="lexeme",how="left")
#derives=derives.merge(autresDerives,on="lexeme",how="left")

In [75]:
derives

,ortho,phono,ext,cs,ms,vs,lexeme,freq,base
0,absoute,absut,@,N,FS,NaN,absoute,14000000,absoudre
1,absoutes,absut,"@z""",N,FP,NaN,absoute,14,absoudre
2,accalmie,akalmi,NaN,N,FS,NaN,accalmie,343000000,accalmir
3,accalmies,akalmi,"z""",N,FP,NaN,accalmie,78000000,accalmir
4,accrue,akry,NaN,N,FS,NaN,accrue,7000000,accroître
5,accrues,akry,"z""",N,FP,NaN,accrue,7,accroître
6,agonie,agòni,NaN,N,FS,NaN,agonie,1782000000,agonir
7,agonies,agòni,"z""",N,FP,NaN,agonie,147000000,agonir
8,allée,ale,NaN,N,FS,NaN,allée,4597000000,aller
9,allées,ale,"z""",N,FP,NaN,allée,2855000000,aller


### Ajout d'une colonne *prob* pour la fréquence relative

In [76]:
derives["prob"]=derives["freq"]/derives["freq"].sum()

### Ajout d'une colonne *case*
- remplir les *ms* vides pour permettre la concaténation sans erreur
- remplir les *vs* vides des participes passés par "pp" pour permettre l'identification de la case
- *case* est la concaténation de *vs* et *ms*

In [78]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

derives["phono"]=derives["phono"].apply(recoder)

In [95]:
with open("2015-Data/"+filePrefix+'-Derives.pkl', 'wb') as output:
   pickle.dump(derives, output, pickle.HIGHEST_PROTOCOL)

In [90]:
derives.sort_values(by="prob")

,ortho,phono,ext,cs,ms,vs,lexeme,freq,base,prob
149,pavées,pave,"z""",N,FP,NaN,pavée,1.000000e-02,paver,1.843121e-14
68,déprise,dEpriz,@,N,FS,NaN,déprise,1.000000e-02,déprendre,1.843121e-14
69,déprises,dEpriz,"@z""",N,FP,NaN,déprise,1.000000e-02,déprendre,1.843121e-14
146,passée,pase,NaN,N,FS,NaN,passée,1.000000e-02,passer,1.843121e-14
147,passées,pase,"z""",N,FP,NaN,passée,1.000000e-02,passer,1.843121e-14
148,pavée,pave,NaN,N,FS,NaN,pavée,1.000000e-02,paver,1.843121e-14
180,reconduite,r6kôdHit,@,N,FS,NaN,reconduite,1.000000e-02,reconduire,1.843121e-14
181,reconduites,r6kôdHit,"@z""",N,FP,NaN,reconduite,1.000000e-02,reconduire,1.843121e-14
198,reposée,r6pOze,NaN,N,FS,NaN,reposée,1.000000e-02,reposer,1.843121e-14
199,reposées,r6pOze,"z""",N,FP,NaN,reposée,1.000000e-02,reposer,1.843121e-14


In [84]:
tirage=np.random.choice(derives.index,size=100,p=derives["prob"],replace=False)

In [ ]:
extrait=derives.ix[np.sort(tirage)]

In [92]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['ms'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
paradigmes

ms,lexeme,FP,FS
0,agonie,NaN,agOni
1,allée,ale,ale
2,arrivée,NaN,arive
3,assise,asiz,NaN
4,attente,atât,atât
5,buée,NaN,bye
6,conduite,NaN,kôdHit
7,contrainte,NaN,kôtrêt
8,crainte,krêt,krêt
9,descente,NaN,dEsât


In [94]:
paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+"-derives.csv",encoding="utf8",sep=";")

In [97]:
GOLD=pd.pivot_table(derives, values='phono', index=['lexeme'], columns=['ms'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
GOLD.to_csv(path_or_buf="2015-Data/"+filePrefix+"-derives-GOLD.csv",encoding="utf8",sep=";")